In [2]:
import requests
import pandas as pd
import os

# Set your save path
save_path = r"C:\Users\acer\OneDrive\Desktop\chameleon last sem project"

def fetch_charging_stations(region='AU'):
    url = "https://api.openchargemap.io/v3/poi/"
    params = {
        'output': 'json',
        'countrycode': region,
        'maxresults': 500,
        'compact': True,
        'verbose': False,
        'key': '  # Your API key
    }

    try:
        response = requests.get(url, params=params)
        print(f"Status Code: {response.status_code}")

        if response.status_code != 200:
            print("❌ Failed to fetch data. Response:")
            print(response.text)
            return pd.DataFrame()

        data = response.json()

        if not data:
            print("⚠️ No data returned from the API.")
            return pd.DataFrame()

        df = pd.json_normalize(data)
        csv_path = os.path.join(save_path, 'charging_stations.csv')
        df.to_csv(csv_path, index=False)
        print(f"✅ Data saved to: {csv_path}")
        return df

    except requests.exceptions.RequestException as e:
        print("❌ Request error:", e)
        return pd.DataFrame()

    except ValueError as e:
        print("❌ JSON decoding error:", e)
        print("Raw response text:", response.text)
        return pd.DataFrame()

# Call the function
charging_df = fetch_charging_stations()



Status Code: 200
✅ Data saved to: C:\Users\acer\OneDrive\Desktop\chameleon last sem project\charging_stations.csv


In [3]:
import pandas as pd

# Load data
chargers = pd.read_csv(os.path.join(save_path, 'charging_stations.csv'))
# Additional datasets: amenities, census etc.
# Merge/Join by proximity using Geopandas or Haversine formula


In [6]:
import requests
import pandas as pd
import os
import numpy as np

save_path = r"C:\Users\acer\OneDrive\Desktop\chameleon last sem project"
os.makedirs(save_path, exist_ok=True)  # Ensure the folder exists

def fetch_charging_stations(region='AU'):
    url = "https://api.openchargemap.io/v3/poi/"
    params = {
        'output': 'json',
        'countrycode': region,
        'maxresults': 500,
        'compact': True,
        'verbose': False,
        'key': 'b2c464d0-a51a-4e70-954d-4bc96acb2f17'
    }

    try:
        response = requests.get(url, params=params)
        print(f"Status Code: {response.status_code}")

        if response.status_code != 200:
            print("❌ Failed to fetch data. Response:")
            print(response.text)
            return pd.DataFrame()

        data = response.json()

        if not data:
            print("⚠️ No data returned from the API.")
            return pd.DataFrame()

        df = pd.json_normalize(data)

        # TEMP file name to avoid file lock issues
        csv_path = os.path.join(save_path, 'charging_stations_temp.csv')
        df.to_csv(csv_path, index=False)
        print(f"✅ Data saved to: {csv_path}")
        return df

    except requests.exceptions.RequestException as e:
        print("❌ Request error:", e)
        return pd.DataFrame()

    except ValueError as e:
        print("❌ JSON decoding error:", e)
        print("Raw response text:", response.text)
        return pd.DataFrame()

# Step 1: Fetch charging station data
chargers = fetch_charging_stations()

# Step 2: Create mock columns and AIS
if not chargers.empty:
    chargers['DistanceToMainRoads'] = np.random.rand(len(chargers))
    chargers['DistanceToAmenities'] = np.random.rand(len(chargers))
    chargers['WalkabilityScore'] = np.random.rand(len(chargers))
    chargers['WheelchairAccessible'] = np.random.choice([0, 1], size=len(chargers))
    chargers['NearbyPopulationDensity'] = np.random.rand(len(chargers))

    chargers['ais_score'] = (
        0.2 * chargers['DistanceToMainRoads'] +
        0.2 * chargers['DistanceToAmenities'] +
        0.2 * chargers['WalkabilityScore'] +
        0.2 * chargers['WheelchairAccessible'] +
        0.2 * chargers['NearbyPopulationDensity']
    )

    chargers['ais_score'] = 1 - chargers['ais_score']

    # Save final file with different name to avoid locked issues
    output_path = os.path.join(save_path, 'chargers_with_ais_temp.csv')
    chargers.to_csv(output_path, index=False)
    print(f"✅ AIS scores calculated and saved to: {output_path}")
else:
    print("⚠️ No data to process.")


Status Code: 200
✅ Data saved to: C:\Users\acer\OneDrive\Desktop\chameleon last sem project\charging_stations_temp.csv
✅ AIS scores calculated and saved to: C:\Users\acer\OneDrive\Desktop\chameleon last sem project\chargers_with_ais_temp.csv


In [7]:
from sklearn.cluster import DBSCAN
import numpy as np

coords = chargers[['AddressInfo.Latitude', 'AddressInfo.Longitude']].dropna().values
db = DBSCAN(eps=0.03, min_samples=5).fit(coords)
chargers['cluster'] = db.labels_
chargers.to_csv(os.path.join(save_path, 'clustered_chargers.csv'), index=False)
